In [6]:
# ============================================================
# gensim v4 で「学習済み Word2Vec をロードして使う」ための標準コード
# ------------------------------------------------------------
# 目的:
#   - v3 時代の `Word2Vec.load('xxx')` は「完全モデル（学習器つき）」を
#     Pickle したときだけ有効でした。v4 では “推論だけ使えれば十分”
#     という前提で **KeyedVectors**（語→ベクトル辞書）を使うのが基本です。
#
# 方針:
#   1) まず KeyedVectors のネイティブ形式（.kv）を探す
#   2) つぎに word2vec 互換形式（.vec / .bin）を試す
#   3) どうしても完全モデルを読みたい場合のみ Word2Vec.load（v4保存物に限る）
#
# 実務メモ:
#   - 使う側（類似語検索・ベクトル取得）は KeyedVectors で十分。
#   - v3 で保存した Pickle（.bin など）を v4 で読めない場合が多いので、
#     互換形式（.kv / .vec / .bin〔word2vec互換バイナリ〕）へ変換しておくのが安全策です。
# ============================================================

from pathlib import Path
from gensim.models import KeyedVectors, Word2Vec

# 想定されるファイル候補を列挙（環境に合わせてパスは調整してください）
CANDIDATE_KV  = Path("ja.kv")   # gensim v4 推奨の KeyedVectors 形式
CANDIDATE_VEC = Path("ja.vec")  # word2vec 互換テキスト
CANDIDATE_BIN = Path("ja.bin")  # ① word2vec 互換バイナリ or ②（v4で保存した）完全モデル

def load_w2v_v4() -> KeyedVectors:
    """
    学習済みベクトルを **gensim v4 流儀** でロードして KeyedVectors を返す。
    - .kv があれば最優先
    - .vec（text）があれば load_word2vec_format(binary=False)
    - .bin は 2 通りを試す:
        (A) word2vec 互換バイナリとして KeyedVectors にロード
        (B) v4 で保存した完全モデルとして Word2Vec.load → .wv を取り出す
    """
    # 1) .kv: v4 ネイティブ（最も確実）
    if CANDIDATE_KV.exists():
        kv = KeyedVectors.load(str(CANDIDATE_KV))
        print(f"[OK] KeyedVectors をロード: {CANDIDATE_KV}")
        return kv

    # 2) .vec: word2vec 互換テキスト
    if CANDIDATE_VEC.exists():
        kv = KeyedVectors.load_word2vec_format(str(CANDIDATE_VEC), binary=False)
        print(f"[OK] word2vec 互換テキストをロード: {CANDIDATE_VEC}")
        return kv

    # 3) .bin の扱いは要注意:
    #    (A) まずは「word2vec 互換バイナリ」として読む
    if CANDIDATE_BIN.exists():
        # 3-A) KeyedVectors (binary=True) 経由の読み込み
        try:
            kv = KeyedVectors.load_word2vec_format(str(CANDIDATE_BIN), binary=True)
            print(f"[OK] word2vec 互換バイナリとしてロード: {CANDIDATE_BIN}")
            return kv
        except Exception as e1:
            print(f"[INFO] word2vec 互換としては読めず: {e1}")

        # 3-B) v4 で保存した「完全モデル」として読み、.wv を取り出す
        try:
            model = Word2Vec.load(str(CANDIDATE_BIN))
            kv = model.wv
            print(f"[OK] 完全モデル（v4保存）をロード → KeyedVectors 抽出: {CANDIDATE_BIN}")
            return kv
        except Exception as e2:
            print(f"[INFO] v4 完全モデルとしても読めず: {e2}")

    # ここまで来たら適切な形式が手元に無い
    raise FileNotFoundError(
        "学習済みベクトルを読み込めませんでした。\n"
        "利用可能な形式の例: ja.kv（推奨）, ja.vec（text）, ja.bin（word2vec互換binary or v4保存のWord2Vec）"
    )

# ===== 実行 =====
kv = load_w2v_v4()

# v4 流儀の「同じこと」: モデルの中身（語彙サイズや次元）を表示し、
# 以降は kv（KeyedVectors）経由で利用する
print(f"語彙サイズ: {len(kv):,}")
print(f"ベクトル次元: {kv.vector_size}")

# 代表語の存在チェックとサンプル出力（任意）
probe = "世間"
if probe in kv:
    print(f"\n《{probe} のベクトル先頭10要素》\n{kv[probe][:10]}")
    print(f"\n《{probe} の類似語 Top10》")
    for w, sim in kv.most_similar(probe, topn=10):
        print(f"  {w}\t{sim:.4f}")
else:
    print(f"\n[INFO] 「{probe}」は語彙に未登録です。別語で確認してください。")

# ------------------------------------------------------------
# 補足（理論的説明）
# ------------------------------------------------------------
# - KeyedVectors とは:
#     Word2Vec 学習後の「語→ベクトル」辞書のみを保持する軽量表現。
#     推論（類似語検索・コサイン類似度計算・クラスタリング等）にはこれで十分。
#
# - Word2Vec.load との違い（v4）:
#     * Word2Vec.load は “v4 で保存した完全モデル” を読むときにのみ使う。
#       （学習器・ハイパラ・語彙などを含む。再学習や再学習準備が必要なとき向け）
#     * v3 で保存した Pickle (.bin 等) は v4 では互換性が崩れていることが多いため非推奨。
#
# - 互換形式の使い分け:
#     * .kv …… v4 ネイティブ（最優先）
#     * .vec … word2vec 互換テキスト（可搬性は高いがサイズ大）
#     * .bin … word2vec 互換バイナリ（高速ロード、ただし中身は KeyedVectors のみ）
#
# - 実運用のベストプラクティス:
#     * まずは KeyedVectors（.kv）を配布/保存しておき、必要なら .vec も併置。
#     * v3 遺産の Pickle を見つけたら v4 で一度ロードできる形（.kv / .vec / .bin）に変換しておく。

[OK] KeyedVectors をロード: ja.kv
語彙サイズ: 1,921,410
ベクトル次元: 3

[INFO] 「世間」は語彙に未登録です。別語で確認してください。
